## Blender with Amass

### Set up
    1. Install blender
    2. Install blender SMPL-X add-on: https://gitlab.tuebingen.mpg.de/jtesch/smplx_blender_addon
    2. Create a conda environment with the same python version as blender 
        (You can check it at blender's python interactive console under scripting tab)
    3. In that environment, install blender_notebook: https://github.com/cheng-chi/blender_notebook
    4. open this script in jupyter notebook and select blender kernel
    5. Run this script

In [1]:
import bpy
import numpy as np
from tqdm import tqdm

In [2]:
# smpl-x skeleton dict
skeleton_dict = { 0: 'pelvis',     3: 'spine1',       6: 'spine2',    9: 'spine3',    12: 'neck',     15: 'head',
     1: 'left_hip',      4: 'left_knee',       7: 'left_ankle',  10: 'left_foot',
     2: 'right_hip',      5: 'right_knee',       8: 'right_ankle',  11: 'right_foot',
    13: 'left_collar',  16: 'left_shoulder',  18: 'left_elbow',  20: 'left_wrist',
    14: 'right_collar',  17: 'right_shoulder',  19: 'right_elbow',  21: 'right_wrist',
    22: 'jaw',      23: 'left_eye_smplhf',    24: 'right_eye_smplhf',
    25: 'left_index1',  26: 'left_index2',  27: 'left_index3',
    28: 'left_middle1', 29: 'left_middle2', 30: 'left_middle3',
    31: 'left_pinky1',  32: 'left_pinky2',  33: 'left_pinky3',
    34: 'left_ring1',   35: 'left_ring2',   36: 'left_ring3',
    37: 'left_thumb1',  38: 'left_thumb2',  39: 'left_thumb3',
    40: 'right_index1',  41: 'right_index2',  42: 'right_index3',
    43: 'right_middle1', 44: 'right_middle2', 45: 'right_middle3',
    46: 'right_pinky1',  47: 'right_pinky2',  48: 'right_pinky3',
    49: 'right_ring1',   50: 'right_ring2',   51: 'right_ring3',
    52: 'right_thumb1',  53: 'right_thumb2',  54: 'right_thumb3'}

In [3]:
# Add a model into scene
bpy.data.window_managers["WinMan"].smplx_tool.smplx_gender = "neutral"
bpy.ops.scene.smplx_add_gender()

Adding gender: neutral
Setting hand pose: relaxed


/snap/blender/830/2.93/python/lib/python3.9/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'FINISHED'}

In [12]:
def get_animation_data(data_path: str):
    data = np.load(data_path, allow_pickle=True)
    for item in data.files:
        print(item)
        print(data[item])
    total_frame = int(data["mocap_time_length"] * data["mocap_frame_rate"])
    return data, total_frame
    
def set_animation(data, total_frame):
    # set frame properties
    bpy.data.scenes["Scene"].frame_start = 0
    bpy.data.scenes["Scene"].frame_end = total_frame
    bpy.data.objects['SMPLX-neutral'].select_set(True)
    bpy.ops.object.mode_set(mode="POSE")
    bpy.context.scene.render.fps = round(float(data["mocap_frame_rate"]))
    
    # change rotation model to xyz-euler
    character = bpy.data.objects["SMPLX-neutral"]
    root = character.pose.bones["root"]
    root.rotation_mode = "XYZ"
    for i, joint_name in skeleton_dict.items():
        character.pose.bones[joint_name].rotation_mode = "XYZ"
        
    for frame in tqdm(range(total_frame)):
        # set root location and rotation
        root.location = data["trans"][frame]
        root.keyframe_insert(data_path="location", frame=frame)
        root.rotation_euler = data["root_orient"][frame]
        root.keyframe_insert(data_path="rotation_euler", frame=frame)
        
        # set skeleton joints
        for i, joint_name in skeleton_dict.items():
            joint = character.pose.bones[joint_name]
            joint.rotation_euler = data["poses"][frame][i * 3: (i + 1) * 3]
            joint.keyframe_insert(data_path="rotation_euler", frame=frame)            

In [13]:
data, total_frame = get_animation_data("/home/vince/Documents/Learning/SSM/20161014_50033/jumping_jacks_sync_stageii.npz")

gender
neutral
surface_model_type
smplx
mocap_frame_rate
120.00004577636719
mocap_time_length
2.3916657543185855
markers_latent
[[ 1.93145859e-03  1.26610943e-01 -9.33853262e-02]
 [ 6.21955528e-03  9.06648320e-02  4.07210988e-02]
 [ 1.23748289e-01 -1.12044955e+00 -7.55488377e-02]
 [ 6.84035007e-02  1.10225084e-01 -1.08061075e-01]
 [ 2.64208311e-01  8.56375080e-02 -4.28345760e-03]
 [ 5.77440152e-02  2.56632436e-01 -8.28564945e-02]
 [ 6.79398222e-02 -3.86816835e-01 -1.42634413e-01]
 [ 1.05794040e-01 -4.65375285e-02  1.00538645e-01]
 [ 7.71172547e-02 -2.74029649e-01 -1.04430138e-01]
 [ 5.41454695e-02  2.28052168e-01  7.31746026e-02]
 [ 3.77295582e-01  9.32710509e-02 -4.28382240e-02]
 [ 3.75020904e-01  9.93705134e-03 -1.35637311e-02]
 [ 7.34460035e-02  3.08843616e-01  3.17668975e-02]
 [ 6.55064791e-01  4.96093785e-02 -9.20510014e-02]
 [ 4.85016851e-01  8.57375240e-02 -4.87438951e-02]
 [ 7.32337390e-02 -6.42997182e-01  6.23499900e-02]
 [ 1.09681899e-01 -2.82201920e-01  7.06493018e-02]
 [ 9.

In [17]:
set_animation(data, total_frame)

100%|█████████████████████████████████████████| 287/287 [00:08<00:00, 33.37it/s]


In [18]:
# Clear all animation
bpy.context.active_object.animation_data_clear()